In [ ]:
import sys
sys.path.append('/opt/anaconda3/lib/python38.zip')
sys.path.append('/opt/anaconda3/lib/python3.8')
sys.path.append('/opt/anaconda3/lib/python3.8/lib-dynload')
sys.path.append('/home/oytj/.local/lib/python3.8/site-packages')
sys.path.append('/opt/anaconda3/lib/python3.8/site-packages')

In [ ]:
import librosa
import librosa.display
from matplotlib import pyplot as plt 
import IPython
import math
import numpy as np
import os
import random
import time
import numpy as np
import cv2 as cv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable
import pyworld
import pysptk
import model

## 数据预处理
### 读入干净的长音频，重采样为16384,分割4s小段

In [ ]:
if False:
    clean_voc_seg=[]
    sr16k=16384
    path='./data/train/wav'
    savePath='./data/train/sr16k'
    filenames=os.listdir(path)
    filenames.sort()
    fileHead=""
    temp_voc_seg=[]
    for i in range(len(filenames)): 
        filename=filenames[i]
        if filename[-4:]=='.wav':
            if (fileHead!=filename.split('_')[0]):
                if len(temp_voc_seg)>0:
                    clean_voc_seg.append(np.array(temp_voc_seg))
                    saveName='speaker_'+fileHead+'.npy'
                    print('\nSave',saveName)
                    np.save(savePath+'/'+saveName,np.array(temp_voc_seg))
                    temp_voc_seg=[]
                fileHead=filename.split('_')[0]
                    #if(i>2000):
                    #    break
            print('[{}/{}]reading     {}...     '.format(i+1,len(filenames),filename),end='\r' )
            src_voc, sr = librosa.load(path+'/'+filename,sr=None)#读取
            print('[{}/{}]resampleing {}...     '.format(i+1,len(filenames),filename),end='\r' )
            src_voc_16k=librosa.resample(src_voc,sr,sr16k)#重采样
            print('[{}/{}]spliting    {}...     '.format(i+1,len(filenames),filename),end='\r' )
            seconds=math.floor(len(src_voc_16k)/sr16k)
            for j in range(0,seconds-4,5):
                temp_voc_seg.append(src_voc_16k[j*sr16k:(j+4)*sr16k])
        if len(temp_voc_seg)>0:
            clean_voc_seg.append(np.array(temp_voc_seg))
            saveName='speaker_'+fileHead+'.npy'
            #print('\nSave',saveName)
            np.save(savePath+'/'+saveName,np.array(temp_voc_seg))    


In [ ]:
#读取保存的音频array，节约时间
clean_voc_seg=[]
savePath='./data/train/sr16k'
filenames=os.listdir(savePath)
filenames.sort()
for i in range(len(filenames)): 
    filename=filenames[i]
    print('[{}/{}]reading     {}...     '.format(i+1,len(filenames),filename),end='\r' )
    read_arr = np.load(savePath+'/'+filename)
    clean_voc_seg.append(read_arr)

In [ ]:
print(len(clean_voc_seg))
clean_voc_seg[1].shape

### 读入噪声

In [ ]:
noise_seg=[]
sr16k=16384
path=r'./data/train/noise'
for subname in os.listdir(path):
    subpath=path+'/'+subname
    for filename in os.listdir(subpath):
        if filename[-4:]=='.wav':
            print('reading '+filename+'...   ',end='\r' )
            n, sr = librosa.load(subpath+'/'+filename,sr=None)
            n16k=librosa.resample(n,sr,sr16k)
            if len(n16k)>5*sr16k:
                noise_seg.append(n16k[sr16k:5*sr16k])
for i in range(len(noise_seg)):
    stdMax=1.0
    srcMax=noise_seg[i].max()
    noise_seg[i]=noise_seg[i]*stdMax/srcMax
noise_seg=np.array(noise_seg)
noise_seg.shape

### 时频转换与显示

In [ ]:
N_FFT = 512
def wav2spectrum(wav):
    S = librosa.stft(wav, N_FFT)
    p = np.angle(S)
    S = np.log1p(np.abs(S))
    return S

def spectrum2wav(spectrum):
    a = np.exp(spectrum) - 1
    p = 2 * np.pi * np.random.random_sample(spectrum.shape) - np.pi
    for i in range(50):
        S = a * np.exp(1j * p)
        x = librosa.istft(S)
        p = np.angle(librosa.stft(x, N_FFT))
    return x

def disWavSpec(wav,title='wavform',colorbar=False):
    spec=wav2spectrum(wav)
    plt.figure(figsize=(10, 14))
    plt.title(title)
    plt.subplot(2,1,1),librosa.display.waveplot(wav, sr=sr16k),plt.xticks([]), plt.yticks([])
    plt.subplot(2,1,2),librosa.display.specshow(spec, sr=sr),plt.xticks([]), plt.yticks([])
    #plt.subplot(2,1,2),librosa.display.specshow(spec, sr=sr, x_axis='time', y_axis='mel'),plt.xticks([]), plt.yticks([])
    if colorbar:
        plt.colorbar()
    plt.show()
    return IPython.display.Audio(wav,rate=sr16k)

### 训练数据生成

In [ ]:
def spectrumOpt(spectrum,opt='',kernelSize=(3,3)):
    if opt=='':
        opt=random.randint(0,7)
    if(opt==0):#平均滤波
        dst = cv.blur(spectrum,kernelSize)
    elif(opt==1):#中值滤波
        dst = cv.medianBlur(spectrum,kernelSize[0])
    elif(opt==2):#高斯滤波
        dst = cv.GaussianBlur(spectrum,kernelSize,0)
    elif(opt==3):#双边滤波
        dst = cv.bilateralFilter(spectrum,kernelSize[0],75,75)
    elif(opt==4):#腐蚀
        kernel = np.ones(kernelSize,np.uint8)
        dst = cv.erode(spectrum,kernel,iterations = 1)
    elif(opt==5):#膨胀
        kernel = np.ones(kernelSize,np.uint8)
        dst = cv.dilate(spectrum,kernel,iterations = 1)
    elif(opt==6):#开
        kernel = np.ones(kernelSize,np.uint8)
        dst = cv.morphologyEx(spectrum,cv.MORPH_OPEN,kernel)
    elif(opt==7):#闭
        kernel = np.ones(kernelSize,np.uint8)
        dst = cv.morphologyEx(spectrum,cv.MORPH_CLOSE,kernel)
    return dst
#####################################################
#增加全局变量用来替代random
#干净声音集合，噪声集合，batchSize，是否为训练数据，指定的演讲者，指定的噪声，是否使用图形学处理混合音频频谱
def getData(clean_voc_seg,noise_seg,batch_size,Training=True,Speaker='',Noise='',mixedAudioOpt=True):
    noise=None
    style=None
    label=None
    split=int((len(clean_voc_seg))*0.85)
    for i in range(batch_size):
        #随机抽取音频数据、风格数据、噪声数据
        if Speaker!='':
            speakerNo=Speaker[0]
            datNo=Speaker[1]
            styNo=Speaker[2]
        else:
            if Training==True:
                speakerNo=random.randint(0,split-1)
            else:
                speakerNo=random.randint(split,len(clean_voc_seg)-1)
            datNo,styNo=random.sample(range(0,len(clean_voc_seg[speakerNo])-1),2)

        #合成音频与噪声,噪声强度随机
        if Noise!='':
            noiseNo=Noise[0]
            noiseAm=Noise[1]
        else:
            noiseNo=random.randint(0,len(noise_seg)-1)
            noiseAm=random.uniform(0.0,0.3)
        w_noise=clean_voc_seg[speakerNo][datNo]+(noise_seg[noiseNo]*noiseAm)
        w_style=clean_voc_seg[speakerNo][styNo]
        w_label=clean_voc_seg[speakerNo][datNo]
        s_noise=wav2spectrum(w_noise)
        if mixedAudioOpt==True:
            s_noise=spectrumOpt(s_noise)
        elif type(mixedAudioOpt)==type(0):
             s_noise=spectrumOpt(s_noise,opt=mixedAudioOpt)   
        s_style=wav2spectrum(w_style)
        s_label=wav2spectrum(w_label)
        t_noise=torch.from_numpy(s_noise.reshape(1,1,s_noise.shape[0],s_noise.shape[1]))
        t_style=torch.from_numpy(s_style.reshape(1,1,s_style.shape[0],s_style.shape[1]))
        t_label=torch.from_numpy(s_label.reshape(1,1,s_label.shape[0],s_label.shape[1]))
        if(noise==None):
            noise=t_noise
            style=t_style
            label=t_label
        else:
            noise=torch.cat((noise, t_noise), dim=0)
            style=torch.cat((style, t_style), dim=0)
            label=torch.cat((label, t_label), dim=0)
    if torch.cuda.is_available():
        mixedAudio=noise.cuda()
        style=style.cuda()
        label=label.cuda()
    #返回：带噪声的音频，风格模板，干净的音频，噪音编号，（说话者编号，讲话内容编号，风格音频编号，噪声编号）
    return mixedAudio,style,label,(speakerNo,datNo,styNo,noiseNo)

## 训练

In [ ]:
loadNet=True
loadNetName='./trainedModel/Gan4-best.pth'
saveNetPath='./trainedModel/'
train_loss_set=[]
val_loss_set=[]
val_Match_set=[]
#新建网络
discriminator=model.Discriminator()
generator=model.Generator()
optimizerG = torch.optim.Adam(generator.parameters(), lr=0.001) # optimizer 使用 Adam
#optimizerG=torch.optim.RMSprop(generator.parameters(), lr=0.0001)
g_lambda = 10  # regularizer for generator
optimizerD = torch.optim.Adam(discriminator.parameters(), lr=0.0001)
#optimizerD = torch.optim.RMSprop(discriminator.parameters(), lr=0.0001)
if loadNet==True:
    #从文件读取网络
    savedNet=torch.load(loadNetName)
    generator.load_state_dict(savedNet['netG'])
    discriminator.load_state_dict(savedNet['netD'])
    #optimizerG.load_state_dict(savedNet['optimizerG'])
    #optimizerD.load_state_dict(savedNet['optimizerD'])
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    discriminator=discriminator.cuda()
    generator=generator.cuda()
loss=nn.L1Loss()
minLossID=-1.0
minLossVar=100.0
totle_epoch=1247####接着上一次
d_update=True
def setNewRange(srcTensor,stdTensor):
    for i in range(srcTensor.size()[0]):
        stdMax=float(torch.max(stdTensor[i]))
        srcMax=float(torch.max(srcTensor[i]))
        srcTensor[i]=srcTensor[i]*stdMax/srcMax


In [ ]:
def train(batch_size,d_epoch_size,g_epoch_size):    
    global discriminator
    global generator
    global optimizerG
    global g_lambda
    global optimizerD
    global minLossID
    global minLossVar
    global totle_epoch
    global d_update
    torch.cuda.empty_cache()
    epoch_start_time = time.time()
    discriminator.train()
    generator.train()
    train_loss = 0.0
    val_loss = 0.0
    dtLoss=0.0
    for i in range(d_epoch_size):
        if d_update==False:
            break
        xn,xs,yl,_=getData(clean_voc_seg,noise_seg,batch_size,Training=True)#抽取训练数据
        #训练Discriminator
        discriminator.zero_grad()
        # TRAIN D to recognize generated audio as noisy
        generated_outputs = generator(xn, xs)
        outputs = discriminator(generated_outputs,xn)
        noisy_loss = 10*torch.mean(outputs ** 2)  # L2 loss - we want them all to be 0
        noisy_loss.backward()
        
        # TRAIN D to recognize clean audio as clean
        setNewRange(yl,generated_outputs)
        outputs = discriminator(yl,xn)
        clean_loss = 10*torch.mean((outputs - 1.0) ** 2)  # L2 loss - we want them all to be 1
        clean_loss.backward()
        optimizerD.step()
        dtLoss=dtLoss+noisy_loss+clean_loss
        print("1.D:batch {:0>3d}/{:0>3d} in epoch {:0>3d}/{:0>3d} Loss={:.10f}".format(i+1,d_epoch_size,epoch+1,epochs,(clean_loss+noisy_loss)/batch_size),end='\r')
    dtLoss/=(d_epoch_size*batch_size)
    '''
    if dtLoss>0.05:
        print('[%03d/%03d-%03d] %3.2f sec(s) dtLoss:%3.6f Skip Train Generater' % \
            (epoch + 1, epochs,totle_epoch, time.time()-epoch_start_time, \
             dtLoss),'                    '\
         )
        return
    '''
    sum_g_loss_=0
    for i in range(g_epoch_size):
        #训练Generator
        xn,xs,yl,_=getData(clean_voc_seg,noise_seg,batch_size,Training=True)#抽取训练数据
        # TRAIN G so that D recognizes G(z) as real
        generator.zero_grad()
        generated_outputs = generator(xn, xs)
        setNewRange(yl,generated_outputs)
        outputs = discriminator(generated_outputs,xn)
        _g_loss_ = 0.5 * torch.mean((outputs - 1.0) ** 2)
        sum_g_loss_+=_g_loss_.item()
        
        l1_loss=loss(generated_outputs, yl)
        g_loss=( _g_loss_ + g_lambda * l1_loss)*2
        g_loss.backward()
        optimizerG.step()
        
        #batch_loss = loss(generated_outputs, yl)
        train_loss += g_loss.item()
        print('2.G:batch {:0>3d}/{:0>3d} in epoch {:0>3d}/{:0>3d} G_loss={:.10f} L1_loss{:.10f}'.format(i+1,g_epoch_size,epoch+1,epochs,_g_loss_/batch_size,l1_loss/batch_size),end='\r')
    sum_g_loss_/=(g_epoch_size*batch_size)
    state={
        'netG':generator.state_dict(),
        'optimizerG':optimizerG.state_dict(),
        'netD':discriminator.state_dict(),
        'optimizerD':optimizerD.state_dict(),
    }
    torch.save(state,saveNetPath+ 'Gan4-last.pth')
    
    if sum_g_loss_>0.05:
        d_update=False
    else:
        d_update=True
        state={
            'netG':generator.state_dict(),
            'optimizerG':optimizerG.state_dict(),
            'netD':discriminator.state_dict(),
            'optimizerD':optimizerD.state_dict(),
        }
        torch.save(state, saveNetPath+'Gan4-best.pth')
    trainTime=time.time()-epoch_start_time
    print('[%03d/%03d-%03d] %3.2f sec(s) Train Loss: %3.6f |d_update:' % \
            (epoch + 1, epochs,totle_epoch, trainTime, \
            train_loss/(batch_size*g_epoch_size)),\
            d_update,sum_g_loss_,'       '
         )
    totle_epoch+=1  
    #测试
    if epoch%100 == 0:
        testLoss=0.00
        for i in range(10):
            xn,xs,yl,_=getData(clean_voc_seg,noise_seg,batch_size,Training=False)#抽取测试数据
            generator.zero_grad()
            generated_outputs = generator(xn, xs)
            setNewRange(yl,generated_outputs)
            outputs = discriminator(generated_outputs,xn)
            _g_loss_ = 0.5 * torch.mean((outputs - 1.0) ** 2)
            l1_loss=loss(generated_outputs, yl)
            g_loss=( _g_loss_ + g_lambda * l1_loss)*2
            testLoss += g_loss.item()
        print('%3.2f sec(s) Test Loss: %3.6f ' % \
            (time.time()-trainTime, testLoss/(batch_size*10)))
            
    torch.cuda.empty_cache()


In [ ]:
batch_size=5  #5
d_epoch_size=1 #每个世代训练的batchs
g_epoch_size=30 #
epochs = 3000 
for epoch in range(epochs):
    train(batch_size,d_epoch_size,g_epoch_size)
    torch.cuda.empty_cache()